In [ ]:


import os, re, asyncio
import pandas as pd
from telethon import TelegramClient


api_id = ***
api_hash = ***
phone = ***


CHANNEL_USERNAME = "kunuz"
OUTPUT_FILE = "Uzbek_Latin_400.txt"
TARGET_SENTENCES = 400
MAX_MESSAGES = 10_000


def clean_text(text):
    if not text:
        return ""

    
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", "", text)
    
    text = re.sub(r"[^\w\s.,!?'-]", " ", text)
    
    text = re.sub(r"\+?\d[\d\-\s]{5,}", "", text)
    
    text = re.sub(r"\s+", " ", text).strip()

    
    bad_words = [
        "tomosha qiling", "tomoshga ailing", "abon bo'ling",
        "videoni ko'ring", "video", "kanalimiz", "telegram",
        "rasmiy sahifa", "subscribe", "t.me", "instagram", "facebook",
        ".uz", ".kr"
    ]
    for bw in bad_words:
        text = re.sub(bw, "", text, flags=re.IGNORECASE)

    return text.strip()

def is_valid_message(msg):
    if not msg or len(msg) < 10:
        return False
    if re.search(r"http|www|t\.me|instagram|facebook|youtube|\.uz|\.kr", msg, re.I):
        return False
    
    if re.search(r"[А-Яа-яЁёҚқҒғҲҳЎў]", msg):
        return False
    return True


async def fetch_messages():
    client = TelegramClient("uzbek_latin_session", api_id, api_hash)
    await client.start(phone)
    print("✅ Signed in successfully!")
    print(f"📥 Fetching messages from {CHANNEL_USERNAME}...")

    sentences = []
    async for msg in client.iter_messages(CHANNEL_USERNAME, limit=MAX_MESSAGES):
        if msg.message:
            text = msg.message.strip()
            if is_valid_message(text):
                cleaned = clean_text(text)
                if cleaned and cleaned not in sentences:
                    sentences.append(cleaned)
                    if len(sentences) >= TARGET_SENTENCES:
                        break

    await client.disconnect()
    print(f"💾 Saved {len(sentences)} sentences to {OUTPUT_FILE}")

    pd.Series(sentences).to_csv(
        OUTPUT_FILE, index=False, header=False, encoding="utf-8"
    )


import nest_asyncio
nest_asyncio.apply()
await fetch_messages()


#Please enter the code you received:  ***
#Signed in successfully as ***; remember to not break the ToS or you will risk an account ban!
#✅ Signed in successfully!
#📥 Fetching messages from kunuz...
#💾 Saved 400 sentences to Uzbek_Latin_400.txt